In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, TensorDataset, DataLoader
import random
from torch import Tensor
import math
from torch.nn import TransformerDecoder, TransformerDecoderLayer

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
seq_len = 20

In [ ]:
def generate_palindromic_binary_data(num_samples=1000, length=20):
    data = []
    for _ in range(num_samples):
        half = np.random.randint(0, 2, size=length // 2)
        palindrome = np.concatenate((half, half[::-1]))
        data.append(palindrome)
    return torch.tensor(data, dtype=torch.float32)
def is_palindrome(seq):
    return seq == seq[::-1]

In [ ]:
# Create a DataLoader
batch_size = 32
binary_palindromes = generate_palindromic_binary_data()
train_loader = DataLoader(TensorDataset(binary_palindromes), batch_size=batch_size, shuffle=True)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 20):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

In [ ]:
# Transformer Class, decoder only attempt
class Transformer(nn.Module):
    def __init__(self, vocab_size=2, d_model=64, nhead=2, d_hid=256, num_layers=1, dropout=0.2):
        super(Transformer, self).__init__()
        self.embedding = nn.Embedding(vocab_size + 1, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        decoder_layers = TransformerDecoderLayer(d_model, nhead, d_hid, dropout)
        self.transformer_decoder = TransformerDecoder(decoder_layers, num_layers)
        self.d_model = d_model
        self.linear = nn.Linear(d_model, vocab_size)
        self.vocab_size = vocab_size

    def forward(self, x):
        x = self.embedding(x) * math.sqrt(self.d_model)  
        x = self.pos_encoder(x)
        mask = self.generate_mask(x.size(0)) 
        x = self.transformer_decoder(x, x, tgt_mask=mask)  # Self-attention?
        x = self.linear(x) 
        return torch.softmax(x, dim=-1) #
    
    def generate_mask(self, size):
        return torch.triu(torch.full((size, size), float('-inf')), diagonal=1)

In [10]:
model = Transformer().to(device)

In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss(ignore_index=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        batch = batch.transpose(0, 1).to(device)  # (seq_len, batch_size)

        optimizer.zero_grad()

        input_seq = batch[:seq_len//2, :].to(device)  # First half (input)
        target_seq = batch[seq_len//2:].to(device)  # Second half (output)

        # Forward pass
        output = model(input_seq).to(device)  # (seq_len//2, batch_size, vocab_size)


        # Compute loss
        loss = criterion(output.view(-1, 2), target_seq.reshape(-1))

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Avg Loss: {total_loss / len(train_loader):.4f}")


Epoch 1, Avg Loss: 0.3876
Epoch 2, Avg Loss: 0.3870
Epoch 3, Avg Loss: 0.3842
Epoch 4, Avg Loss: 0.3856
Epoch 5, Avg Loss: 0.3866
Epoch 6, Avg Loss: 0.3861
Epoch 7, Avg Loss: 0.3826
Epoch 8, Avg Loss: 0.3836
Epoch 9, Avg Loss: 0.3857
Epoch 10, Avg Loss: 0.3831
Epoch 11, Avg Loss: 0.3840
Epoch 12, Avg Loss: 0.3835
Epoch 13, Avg Loss: 0.3818
Epoch 14, Avg Loss: 0.3902
Epoch 15, Avg Loss: 0.3844
Epoch 16, Avg Loss: 0.3831
Epoch 17, Avg Loss: 0.3864
Epoch 18, Avg Loss: 0.3828
Epoch 19, Avg Loss: 0.3846
Epoch 20, Avg Loss: 0.3846
Epoch 21, Avg Loss: 0.3842
Epoch 22, Avg Loss: 0.3853
Epoch 23, Avg Loss: 0.3835
Epoch 24, Avg Loss: 0.3814
Epoch 25, Avg Loss: 0.3850
Epoch 26, Avg Loss: 0.3916
Epoch 27, Avg Loss: 0.3843
Epoch 28, Avg Loss: 0.3851
Epoch 29, Avg Loss: 0.3839
Epoch 30, Avg Loss: 0.3861
Epoch 31, Avg Loss: 0.3850
Epoch 32, Avg Loss: 0.3853
Epoch 33, Avg Loss: 0.3858
Epoch 34, Avg Loss: 0.3853
Epoch 35, Avg Loss: 0.3842
Epoch 36, Avg Loss: 0.3840
Epoch 37, Avg Loss: 0.3861
Epoch 38, 

Epoch 298, Avg Loss: 0.3876
Epoch 299, Avg Loss: 0.3846
Epoch 300, Avg Loss: 0.3851
Epoch 301, Avg Loss: 0.3859
Epoch 302, Avg Loss: 0.3876
Epoch 303, Avg Loss: 0.3912
Epoch 304, Avg Loss: 0.3855
Epoch 305, Avg Loss: 0.3837
Epoch 306, Avg Loss: 0.3866
Epoch 307, Avg Loss: 0.3993
Epoch 308, Avg Loss: 0.3870
Epoch 309, Avg Loss: 0.3855
Epoch 310, Avg Loss: 0.3823
Epoch 311, Avg Loss: 0.3877
Epoch 312, Avg Loss: 0.3837
Epoch 313, Avg Loss: 0.3913
Epoch 314, Avg Loss: 0.3920
Epoch 315, Avg Loss: 0.3888
Epoch 316, Avg Loss: 0.3856
Epoch 317, Avg Loss: 0.3922
Epoch 318, Avg Loss: 0.3877
Epoch 319, Avg Loss: 0.3910
Epoch 320, Avg Loss: 0.3901
Epoch 321, Avg Loss: 0.3865
Epoch 322, Avg Loss: 0.3853
Epoch 323, Avg Loss: 0.3850
Epoch 324, Avg Loss: 0.3832
Epoch 325, Avg Loss: 0.3816
Epoch 326, Avg Loss: 0.3856
Epoch 327, Avg Loss: 0.3841
Epoch 328, Avg Loss: 0.3831
Epoch 329, Avg Loss: 0.3844
Epoch 330, Avg Loss: 0.3836
Epoch 331, Avg Loss: 0.3906
Epoch 332, Avg Loss: 0.3869
Epoch 333, Avg Loss:

Epoch 591, Avg Loss: 0.3905
Epoch 592, Avg Loss: 0.3855
Epoch 593, Avg Loss: 0.3911
Epoch 594, Avg Loss: 0.3876
Epoch 595, Avg Loss: 0.3886
Epoch 596, Avg Loss: 0.3859
Epoch 597, Avg Loss: 0.3854
Epoch 598, Avg Loss: 0.3929
Epoch 599, Avg Loss: 0.3872
Epoch 600, Avg Loss: 0.3829
Epoch 601, Avg Loss: 0.3863
Epoch 602, Avg Loss: 0.3829
Epoch 603, Avg Loss: 0.3878
Epoch 604, Avg Loss: 0.3851
Epoch 605, Avg Loss: 0.3898
Epoch 606, Avg Loss: 0.3848
Epoch 607, Avg Loss: 0.3861
Epoch 608, Avg Loss: 0.3857
Epoch 609, Avg Loss: 0.3841
Epoch 610, Avg Loss: 0.3900
Epoch 611, Avg Loss: 0.3890
Epoch 612, Avg Loss: 0.3989
Epoch 613, Avg Loss: 0.3906
Epoch 614, Avg Loss: 0.3855
Epoch 615, Avg Loss: 0.3824
Epoch 616, Avg Loss: 0.3857
Epoch 617, Avg Loss: 0.3850
Epoch 618, Avg Loss: 0.3892
Epoch 619, Avg Loss: 0.3857
Epoch 620, Avg Loss: 0.3853
Epoch 621, Avg Loss: 0.3858
Epoch 622, Avg Loss: 0.3844
Epoch 623, Avg Loss: 0.3889
Epoch 624, Avg Loss: 0.3901
Epoch 625, Avg Loss: 0.3855
Epoch 626, Avg Loss:

Epoch 884, Avg Loss: 0.3734
Epoch 885, Avg Loss: 0.3746
Epoch 886, Avg Loss: 0.3726
Epoch 887, Avg Loss: 0.3748
Epoch 888, Avg Loss: 0.3693
Epoch 889, Avg Loss: 0.3730
Epoch 890, Avg Loss: 0.3829
Epoch 891, Avg Loss: 0.3688
Epoch 892, Avg Loss: 0.3793
Epoch 893, Avg Loss: 0.3750
Epoch 894, Avg Loss: 0.3713
Epoch 895, Avg Loss: 0.3708
Epoch 896, Avg Loss: 0.3713
Epoch 897, Avg Loss: 0.3716
Epoch 898, Avg Loss: 0.3739
Epoch 899, Avg Loss: 0.3780
Epoch 900, Avg Loss: 0.3746
Epoch 901, Avg Loss: 0.3923
Epoch 902, Avg Loss: 0.3861
Epoch 903, Avg Loss: 0.3708
Epoch 904, Avg Loss: 0.3689
Epoch 905, Avg Loss: 0.3698
Epoch 906, Avg Loss: 0.3697
Epoch 907, Avg Loss: 0.3726
Epoch 908, Avg Loss: 0.3733
Epoch 909, Avg Loss: 0.3769
Epoch 910, Avg Loss: 0.3793
Epoch 911, Avg Loss: 0.3729
Epoch 912, Avg Loss: 0.3708
Epoch 913, Avg Loss: 0.3704
Epoch 914, Avg Loss: 0.3718
Epoch 915, Avg Loss: 0.3754
Epoch 916, Avg Loss: 0.3706
Epoch 917, Avg Loss: 0.3703
Epoch 918, Avg Loss: 0.3685


In [ ]:
def generate_palindrome(model, start_token, seq_len=20):
    model.eval()
    generated = torch.tensor([[start_token]], device=device)  # Start sequence

    with torch.no_grad():
        for _ in range(seq_len - 1):
            output = model(generated)  
            next_token = output.argmax(dim=-1)[-1, :]  
            generated = torch.cat([generated, next_token.unsqueeze(0)], dim=0)  

    return generated.squeeze().tolist()  

start_token = 1  
gen_seq = generate_palindrome(model, start_token, seq_len=20)
print("Generated Sequence:", gen_seq)


Generated Sequence: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [ ]:
for i in range(10):
    start_token = torch.randint(0, 2, (1,)).item()  
    generated_seq = generate_palindrome(model, start_token, seq_len=20)
    print(f"Sample {i+1}: {generated_seq}")


Sample 1: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sample 2: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 3: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sample 4: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sample 5: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sample 6: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 7: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Sample 8: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 9: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Sample 10: [1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
